In [ ]:
!pip -q install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 76.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 76.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 58.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.0/220.0 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 4.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currentl

In [ ]:
# For setting up the vector db
import chromadb

# For working with our data files
import os
import glob

# For handling our data
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Miscellaneous
import time # to see how long code processes take to run
import textwrap # to pretty print abstracts

In [ ]:
!gdown 1g3K-wlixFxklTSUQNZKpEgN4WNTFTPIZ

Downloading...
From (original): https://drive.google.com/uc?id=1g3K-wlixFxklTSUQNZKpEgN4WNTFTPIZ
From (redirected): https://drive.google.com/uc?id=1g3K-wlixFxklTSUQNZKpEgN4WNTFTPIZ&confirm=t&uuid=f67e8ac1-f792-4ab0-865c-489545abf0e9
To: /content/arxiv_abstracts.zip
100% 7.55G/7.55G [01:44<00:00, 72.4MB/s]


In [ ]:
!unzip arxiv_abstracts.zip

Archive:  arxiv_abstracts.zip
   creating: arxiv_abstracts/
  inflating: __MACOSX/._arxiv_abstracts  
  inflating: arxiv_abstracts/abstracts_21.parquet  
  inflating: arxiv_abstracts/abstracts_1.parquet  
  inflating: arxiv_abstracts/abstracts_8.parquet  
  inflating: arxiv_abstracts/abstracts_17.parquet  
  inflating: arxiv_abstracts/verifyResults.py  
  inflating: __MACOSX/arxiv_abstracts/._verifyResults.py  
  inflating: arxiv_abstracts/abstracts_16.parquet  
  inflating: arxiv_abstracts/abstracts_9.parquet  
  inflating: arxiv_abstracts/abstracts_20.parquet  
  inflating: arxiv_abstracts/abstracts_14.parquet  
  inflating: arxiv_abstracts/abstracts_22.parquet  
  inflating: arxiv_abstracts/abstracts_2.parquet  
  inflating: arxiv_abstracts/abstracts_23.parquet  
  inflating: arxiv_abstracts/abstracts_3.parquet  
  inflating: arxiv_abstracts/params.txt  
  inflating: __MACOSX/arxiv_abstracts/._params.txt  
  inflating: arxiv_abstracts/abstracts_15.parquet  
  inflating: arxiv_abstra

In [ ]:
def load_parquet_file(file_path: str) -> pd.DataFrame:
    return pd.read_parquet(file_path)

data_df = None
i = 0
for file_path in glob.glob(os.path.join('arxiv_abstracts/', 'abstracts_*.parquet')):
    print('Loading data from {}'.format(file_path))
    file_df = load_parquet_file(file_path)

    if data_df is None:
        # initialize the dataframe
        data_df = file_df
    else:
        # append the new data to our dataframe
        data_df = pd.concat([data_df, file_df], ignore_index=True)

    # Use this to limit the amount of data
    # Each file contains 100,000 rows
    i += 1
    if i >= 5:
        break

# A little data prep
data_df = data_df.rename(columns={'embeddings': 'embedding'})
data_df['id'] = data_df.index.astype(str)

Loading data from arxiv_abstracts/abstracts_21.parquet
Loading data from arxiv_abstracts/abstracts_17.parquet
Loading data from arxiv_abstracts/abstracts_11.parquet
Loading data from arxiv_abstracts/abstracts_22.parquet
Loading data from arxiv_abstracts/abstracts_6.parquet


In [ ]:
data_df.head()

,abstract,embedding,doi,id
0,"We study a single, motionless three-dimensio...","[-0.016700998, -0.026170395, -0.0017791224, 0....",cond-mat/9912225,0
1,We argue that effective 1D models of stripes...,"[0.016688664, 0.03121568, 0.013161989, -0.0053...",cond-mat/9912226,1
2,We study numerically the one dimensional fer...,"[0.012090295, 0.013782813, 0.009999216, -0.019...",cond-mat/9912227,2
3,We overview some recent work and present new...,"[0.015418335, -0.005732018, -0.0061773793, 0.0...",cond-mat/9912228,3
4,An english translation of the original work ...,"[0.011050525, 0.009937972, 0.00615069, -0.0062...",cond-mat/9912229,4


In [ ]:
user_liked_paper = data_df[data_df['doi'] == data_df['doi'][0]].iloc[0].to_dict()
print(f"abstract: \n{user_liked_paper['abstract']}")

abstract: 
  We study a single, motionless three-dimensional droplet growing by adsorption
of diffusing monomers on a 2D substrate. The diffusing monomers are adsorbed at
the aggregate perimeter of the droplet with different boundary conditions.
Models with both an adsorption boundary condition and a radiation boundary
condition, as well as a phenomenological model, are considered and solved in a
quasistatic approximation. The latter two models allow particle detachment. In
the short time limit, the droplet radius grows as a power of the time with
exponents of 1/4, 1/2 and 3/4 for the models with adsorption, radiation and
phenomenological boundary conditions, respectively. In the long time limit a
universal growth rate as $[t/\ln(t)]^{1/3}$ is observed for the radius of the
droplet for all models independent of the boundary conditions. This asymptotic
behaviour was obtained by Krapivsky \cite{krapquasi} where a similarity
variable approach was used to treat the growth of a droplet with

In [ ]:
def brute_force_search(user_liked_paper, data_df, num_results=5):
    start_time = time.time()

    data_df['similarity'] = data_df['embedding'].apply(
		    lambda x: cosine_similarity([user_liked_paper['embedding']], [x])[0][0]
	  )

    # Make sure the results don't contain the same paper as the query
    search_results = data_df[data_df['doi'] != user_liked_paper['doi']].sort_values(
		    by='similarity',
		    ascending=False
	  ).head(num_results)

    print(f"Time taken to brute force search: {round(time.time() - start_time, 3)} seconds")

    # Select and reorder columns, reset index
    search_results = search_results[["id", "abstract", "doi"]].reset_index(
		    drop=True
	  )

    return search_results

search_results = brute_force_search(user_liked_paper, data_df, num_results=5)
display(search_results)

Time taken to brute force search: 205.485 seconds


,id,abstract,doi
0,495226,Droplet patterns condensing on solid substra...,1502.00566
1,464461,We expand on a recent study of a lattice mod...,1410.3360
2,484900,We have studied the growth kinetics of isola...,1412.6922
3,153342,In this paper we consider a multiparticle ve...,2205.08930
4,78033,We study spherically symmetric bubble growth...,hep-ph/9512202


In [ ]:
db_client = chromadb.PersistentClient(path="chroma_db")

In [ ]:
collection_name = "paper_abstracts"

if collection_name in db_client.list_collections():
    db_client.delete_collection(name=collection_name)
    print('Dropped existing collection to remake')

abstract_collection = db_client.create_collection(
    name=collection_name,
    # define the distance metric to use when comparing vectors in this collection
    metadata={"hnsw:space": "cosine"})

In [ ]:
batch_size = 1000
for i in range(0, len(data_df), batch_size):
    data_batch = data_df.iloc[i:i+batch_size]
    abstract_collection.add(
        ids=data_batch['id'].tolist(),
        embeddings=data_batch['embedding'].tolist(),
        documents=data_batch['abstract'].tolist(),
        metadatas=data_batch[['doi']].to_dict(orient='records')
    )

In [ ]:
# Perform Semantic Search
# 1. Convert user query to vector embedding
# 2. Find nearest neighbors in ChromaDB
# 3. Return top-k relevant documents
def vector_db_search(user_liked_paper, collection_name, num_results):
    start_time = time.time()
    search_results = abstract_collection.query(
        query_embeddings=[user_liked_paper['embedding']],
        n_results=num_results,
        # Make sure the results don't contain the same paper as the query
        where={"doi": {"$ne": user_liked_paper['doi']}}
    )

    print(f"Time taken to search vector database: {round(time.time() - start_time, 3)} seconds")

    # Create a pandas dataframe from the search results
    search_results_df = pd.DataFrame(search_results["ids"][0], columns=["id"])
    search_results_df["document"] = search_results["documents"][0]
    search_results_df["doi"] = [meta["doi"] for meta in search_results["metadatas"][0]]
    search_results_df = search_results_df.rename(columns={"document": "abstract"})

    return search_results_df


search_results_df = vector_db_search(user_liked_paper, "article_abstracts", 5)
display(search_results_df)

Time taken to search vector database: 4.912 seconds


,id,abstract,doi
0,495226,Droplet patterns condensing on solid substra...,1502.00566
1,464461,We expand on a recent study of a lattice mod...,1410.3360
2,484900,We have studied the growth kinetics of isola...,1412.6922
3,153342,In this paper we consider a multiparticle ve...,2205.08930
4,78033,We study spherically symmetric bubble growth...,hep-ph/9512202
